In [1]:
from numpy.random import seed
seed(3363)
from tensorflow import set_random_seed
set_random_seed(3363)

In [2]:
from keras import regularizers
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, InputLayer, LeakyReLU, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121
from sklearn.metrics import roc_auc_score
from dataset_batch import load_train_data, load_test_data
import tensorflow as tf

Using TensorFlow backend.


In [3]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

class DenseNetModel():

    def __init__(self, input_dim=(224,224,3), output_dim=14, learning_rate=0.00001, epochs=5, drop_out=0.3,
                 freeze=True):

        # parms:
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.drop_out = drop_out
        self.trainable = not freeze

        # Define DenseNet
        self.model = Sequential()

        base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=self.input_dim)

        # Freeze the model first
        base_model.trainable = self.trainable

        self.model.add(base_model)

        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(512))
        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(self.output_dim, activation = 'sigmoid'))

        auc_roc = as_keras_metric(tf.metrics.auc)
        recall = as_keras_metric(tf.metrics.recall)
        precision = as_keras_metric(tf.metrics.precision)

        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae', auc_roc, recall, precision])
        self.model.summary()

        # Image augmentation
        self.core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip=True, 
                              vertical_flip=False, 
                              height_shift_range=0.05, 
                              width_shift_range=0.15, 
                              rotation_range=10, 
                              shear_range=0.2,
                              fill_mode='constant',
                              cval=1,
                              zoom_range=0.3)

    # Data standardization
    def standardize(self, x):
        self.core_idg.fit(x)
        return

    # fit the data
    def fit(self, x, y, vx, vy):
        print ("Start Training model")
        X_train, y_train, X_test, y_test = x, y, vx, vy
        hist = self.model.fit_generator(
            (self.core_idg.flow(X_train, y_train, batch_size = 30)),
            validation_data = self.core_idg.flow(X_test, y_test), epochs=self.epochs)
        print ("Done Training model")
        return hist

    # data did preprocessing            
    def inference(self, x):
        return self.model.predict(x)

    # make predicition
    def score(self, x, y):
        y_pred = self.predict(x)
        return roc_auc_score(y, y_pred, average = "macro")

    # pain data without preprocessing
    def predict(self, x):
        x = self.core_idg.standardize(x)
        return self.inference(x)

    def save_weight(self, path = 'baseline.h5'):
        print ("Start Saving model")
        self.model.save(path)
        print ("Done Saving model")
        return

    def load_weight(self, path = 'baseline.h5'):
        print ("Start Loading model")
        self.model.load_weights(path)
        print ("Done Loading model")
        return

In [ ]:
if __name__ == "__main__":

    model = DenseNetModel(input_dim=(288,288,3), epochs=4, drop_out=0.7, freeze=False)

    X_vali, y_vali = load_train_data(min_cnt=0, max_cnt=1000)
    for itera in range(25):
        for base in range(1000, 10002, 3100):
            X_train, y_train = load_train_data(min_cnt=base, max_cnt=base+3099)
            model.standardize(X_train)
            model.fit(X_train, y_train, X_vali, y_vali)
        print("Iter",itera,"AVG AUC:",model.score(X_vali, y_vali))
        model.save_weight(path="baseline_"+str(itera)+".h5")

    # Saving model
    model.save_weight()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                7182      
Total params: 7,569,486
Trainable params: 7,485,838
Non-trainable params: 83,648
_________________________________________________________________
Loading Training Images and Labels
0.0 % done


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 0 to 1000
(1001, 288, 288, 3) (1001, 14)
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 215s 2s/step - loss: 0.3118 - binary_accuracy: 0.9212 - mean_absolute_error: 0.1001 - auc: 0.5794 - recall: 0.9904 - precision: 0.0475 - val_loss: 0.3160 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0561 - val_a

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 165s 2s/step - loss: 0.1719 - binary_accuracy: 0.9518 - mean_absolute_error: 0.0852 - auc: 0.7161 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.1915 - val_binary_accuracy: 0.9461 - val_mean_absolute_error: 0.0902 - val_auc: 0.7178 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 165s 2s/step - loss: 0.1695 - binary_accuracy: 0.9522 - mean_absolute_error: 0.0840 - auc: 0.7196 - recall: 1.0000 - precision: 0.0490 - val_loss: 0.1874 - val_binary_accuracy: 0.9478 - val_mean_absolute_error: 0.0799 - val_auc: 0.7211 - val_recall: 1.0000 - val_

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 166s 2s/step - loss: 0.1660 - binary_accuracy: 0.9522 - mean_absolute_error: 0.0828 - auc: 0.7434 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.1884 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0796 - val_auc: 0.7440 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 165s 2s/step - loss: 0.1641 - binary_accuracy: 0.9520 - mean_absolute_error: 0.0832 - auc: 0.7446 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.1896 - val_binary_accuracy: 0.9453 - val_mean_absolute_error: 0.1072 - val_auc: 0.7454 - val_recall: 1.0000 - val_

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 167s 2s/step - loss: 0.1637 - binary_accuracy: 0.9524 - mean_absolute_error: 0.0830 - auc: 0.7553 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.1855 - val_binary_accuracy: 0.9487 - val_mean_absolute_error: 0.0867 - val_auc: 0.7556 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 166s 2s/step - loss: 0.1612 - binary_accuracy: 0.9526 - mean_absolute_error: 0.0828 - auc: 0.7560 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.1817 - val_binary_accuracy: 0.9483 - val_mean_absolute_error: 0.0939 - val_auc: 0.7564 - val_recall: 1.0000 - val_

In [ ]:
model = DenseNetModel(input_dim=(288,288,3), epochs=4, drop_out=0.7, freeze=False)
model.load_weight(path="baseline_2.h5")
X_vali, y_vali = load_train_data(min_cnt=0, max_cnt=1000)
for itera in range(3, 25):
    for base in range(1000, 10002, 3100):
        X_train, y_train = load_train_data(min_cnt=base, max_cnt=base+3099)
        model.standardize(X_train)
        model.fit(X_train, y_train, X_vali, y_vali)
    print("Iter",itera,"AVG AUC:",model.score(X_vali, y_vali))
    model.save_weight(path="baseline_"+str(itera)+".h5")

# Saving model
model.save_weight()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                7182      
Total params: 7,569,486
Trainable params: 7,485,838
Non-trainable params: 83,648
_________________________________________________________________
Start Loading model
Done Loading model
Loading Training Images and Labels
0.0 % done


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 0 to 1000
(1001, 288, 288, 3) (1001, 14)
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 213s 2s/step - loss: 0.1671 - binary_accuracy: 0.9519 - mean_absolute_error: 0.0832 - auc: 0.7740 - recall: 0.9904 - precision: 0.0474 - val_loss: 0.2299 - val_binary_accuracy: 0.9344 - val_mean_absolute_error: 0.1311 - val_a

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 163s 2s/step - loss: 0.1632 - binary_accuracy: 0.9522 - mean_absolute_error: 0.0833 - auc: 0.7631 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.1844 - val_binary_accuracy: 0.9481 - val_mean_absolute_error: 0.0770 - val_auc: 0.7640 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 162s 2s/step - loss: 0.1602 - binary_accuracy: 0.9526 - mean_absolute_error: 0.0823 - auc: 0.7653 - recall: 1.0000 - precision: 0.0490 - val_loss: 0.1781 - val_binary_accuracy: 0.9490 - val_mean_absolute_error: 0.0885

...

5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 160s 2s/step - loss: 0.1244 - binary_accuracy: 0.9574 - mean_absolute_error: 0.0661 - auc: 0.8388 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.2054 - val_binary_accuracy: 0.9451 - val_mean_absolute_error: 0.0701 - val_auc: 0.8389 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 159s 2s/step - loss: 0.1152 - binary_accuracy: 0.9591 - mean_absolute_error: 0.0630 - auc: 0.8390 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.2057 - val_binary_accuracy: 0.9406 - val_mean_absolute_error: 0.0942 - val_auc: 0.8391 - val_recall: 1.0000 - val_

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 161s 2s/step - loss: 0.1237 - binary_accuracy: 0.9572 - mean_absolute_error: 0.0653 - auc: 0.8411 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.2115 - val_binary_accuracy: 0.9330 - val_mean_absolute_error: 0.0960 - val_auc: 0.8412 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 160s 2s/step - loss: 0.1125 - binary_accuracy: 0.9597 - mean_absolute_error: 0.0623 - auc: 0.8414 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.2032 - val_binary_accuracy: 0.9437 - val_mean_absolute_error: 0.0833 - val_auc: 0.8415 - val_recall: 1.0000 - val_

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 4099
(3100, 288, 288, 3) (3100, 14)
Start Training model
Epoch 1/4
104/104 [==============================] - 159s 2s/step - loss: 0.1185 - binary_accuracy: 0.9595 - mean_absolute_error: 0.0634 - auc: 0.8433 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.2074 - val_binary_accuracy: 0.9408 - val_mean_absolute_error: 0.0883 - val_auc: 0.8434 - val_recall: 1.0000 - val_precision: 0.0491
Epoch 2/4
104/104 [==============================] - 160s 2s/step - loss: 0.1070 - binary_accuracy: 0.9620 - mean_absolute_error: 0.0592 - auc: 0.8435 - recall: 1.0000 - precision: 0.0491 - val_loss: 0.2244 - val_binary_accuracy: 0.9346 - val_mean_absolute_error: 0.0945 - val_auc: 0.8436 - val_recall: 1.0000 - val_

In [ ]:
model = DenseNetModel(input_dim=(288,288,3), epochs=4, drop_out=0.7, freeze=False)
model.load_weight(path="baseline_22.h5")
X_vali, y_vali = load_train_data(min_cnt=0, max_cnt=1000)
for itera in range(23, 25):
    for base in range(1000, 10002, 3100):
        X_train, y_train = load_train_data(min_cnt=base, max_cnt=base+3099)
        model.standardize(X_train)
        model.fit(X_train, y_train, X_vali, y_vali)
    print("Iter",itera,"AVG AUC:",model.score(X_vali, y_vali))
    model.save_weight(path="baseline_"+str(itera)+".h5")

# Saving model
model.save_weight()